Esta guia es basada en los tutoriales de la pagina http://tensornetwork.org/ y en el documento de Oseledets, et al https://www.nowpublishers.com/article/Details/MAL-059

In [1]:
!pip install tensornetwork

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA
import tensorflow as tf
import tensornetwork
from tensornetwork import ncon
net = tensornetwork.TensorNetwork()
sess = tf.InteractiveSession()

# Matrix Product State (MPS) o tensor train (TT)

*    Es la factorizacion de un tensor con N indices en un producto tipo cadena lineal de tensores de tres índices. 

*    Es un caso especial de una red tensorial de árbol.

<img src="http://tensornetwork.org/mps/mpstt_diagram.png" width="400">

*    Si MPS/TT es una buena aproximacion de un tensor, entonces podemos reducir el problema de trabajar con un numero de parametros que crece exponencialmente a uno que crece linealmente.   

In [0]:
##### Descomposicion de tensor de 4 indices a MPS usando SVD truncado con ortogonalidad izquierda
d = 50 # dimension externa
H0 = net.add_node(np.random.rand(d,d,d,d)) # 6 indices externos
H0 = net.add_node(H0.tensor/tf.norm(H0.tensor))
deltaval = 0.1

# primera descomposicion
u0,s,vh,trun_err = net.split_node_full_svd(H0,[H0[0]],[H0[1],H0[2],H0[3]],None,deltaval)
H1 = net.add_node(ncon([s.tensor,vh.tensor],[[-1,1],[1,-2,-3,-4]]))

# segunda descomposicion
u1,s,vh,trun_err = net.split_node_full_svd(H1,[H1[0],H1[1]],[H1[2],H1[3]],None,deltaval)
H2 = net.add_node(ncon([s.tensor,vh.tensor],[[-1,1],[1,-2,-3]]))

# tercera descomposicion
u2,s,vh,trun_err = net.split_node_full_svd(H2,[H2[0],H2[1]],[H2[2]],None,deltaval)
H3 = net.add_node(ncon([s.tensor,vh.tensor],[[-1,1],[1,-2]])) 

In [15]:
u2.tensor.shape

TensorShape([Dimension(1549), Dimension(50), Dimension(48)])

In [19]:
H0recovered = net.add_node(ncon([u0.tensor,u1.tensor,u2.tensor,H3.tensor],[[-1,1],[1,-2,2],[2,-3,3],[3,-4]]))
print("totErr = ",(tf.norm(H0recovered.tensor-H0.tensor)/tf.norm(H0.tensor)).eval())

totErr =  0.16879892082524553


# Operaciones elementales con MPS/TT

  
###    Producto interno o superposicion de dos MPS/TT
<img src="http://tensornetwork.org/mps/TdotW.png" width="200">
<br>
<img src="http://tensornetwork.org/mps/TW_MPSTT.png" width="250">
<br>
<img src="http://tensornetwork.org/mps/InnerMPSTT.png" width="250">
  
###    Compresion o redondeo
  
<img src="http://tensornetwork.org/mps/mpstt_compress.png" width="300">